In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import csv
import os
import torch

import random

from PIL import Image
from skimage import io

verbose = False

In [ ]:
class mini_batch(torch.nn.Module):
    def __init__(self, dat_folder,transform,percent):
        super(mini_batch, self).__init__()
        self.root_dir = os.path.dirname(dat_folder) 
        self.transform = transform
        self.percent = percent

        #list of folders 
        ind = [individual for individual in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir,individual))]
        self.ind_Train =  ind[0:int(len(ind)*0.8)] 
        self.ind_Test = ind[int(len(ind)*0.8):-1] 
        
        self.num_ind_train = [len(os.listdir(os.path.join(self.root_dir,i))) for i in self.ind_Train if len(os.listdir(os.path.join(self.root_dir,i)))>3]
        self.num_ind_test = [len(os.listdir(os.path.join(self.root_dir,i))) for i in self.ind_Test if len(os.listdir(os.path.join(self.root_dir,i)))>3]
        
    def loadfile(self, data_file):
        image = io.imread(data_file)
        if len(image.shape)<3:
            image = np.stack((image,)*3, axis=-1)
        return image
    
    
    def prepare_batch(self):

        train_line_list = []
        labels = []

        min_numinf = min(self.num_ind_train)*self.percent
        for i,individual in enumerate(self.ind_Train):
            folder_path = os.path.join(self.root_dir,individual)
            name_images = [name for name in os.listdir(folder_path)]  
 
            N = len(name_images)
            if N>3:
            
                random.shuffle(name_images)


                for j in range(int(min_numinf)):               
                    if os.path.isfile(os.path.join(folder_path,name_images[j])):
                        image_filename = os.path.join(individual,name_images[j])
                        if verbose: print("image {}{}".format(j,image_filename))
                        train_line_list.append(image_filename)
                        labels.append(i)


        image = [self.transform(Image.fromarray(self.loadfile(os.path.join(self.root_dir,train_line_list[i])))) for i in range(len(train_line_list))]                    
        return image,labels
    
    def prepare_batch_test(self):
        test_line_list = []
        labels = []
        min_numinf = min(self.num_ind_test)
        for i,individual in enumerate(self.ind_Test):
            folder_path = os.path.join(self.root_dir,individual)
            name_images = [name for name in os.listdir(folder_path)]
            N = len(name_images)

            if N>3:
                
                for j in range(int(min_numinf)):
                    if os.path.isfile(os.path.join(folder_path,name_images[j])):
                        image_filename = os.path.join(individual,name_images[j])
                        if verbose: print("image {}{}".format(j,image_filename))
                        test_line_list.append(image_filename)
                        labels.append(i)

        image = [self.transform(Image.fromarray(self.loadfile(os.path.join(self.root_dir,test_line_list[i])))) for i in range(len(test_line_list))]                    
        return image,labels          
            

 

In [ ]:
#folder = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Verification/ATeT_faces/orl_faces/"
#dataset_transform = transforms.Compose([
#   transforms.Resize((224,224)),
#   transforms.ToTensor(),        
#   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
#])
#
#image,label = prepare_batch(folder, dataset_transform)
#Variable(image)
